Ref doc link-https://huggingface.co/docs/transformers/tasks/masked_language_modeling#masked-language-modeling

Installing all the required libraries

In [1]:
pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd

Loading wikitext data from huggingface datasets and taking only 10000 instances

In [3]:
from datasets import load_dataset

wikitext_ds = load_dataset(path = "wikitext", name = "wikitext-2-raw-v1", split = "train[:10000]")

wikitext_ds

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 10000
})

Splitting the dataset’s train split into a train and test set with the train_test_split method

In [4]:
wikitext_ds = wikitext_ds.train_test_split(test_size = 0.2)

wikitext_ds

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 2000
    })
})

For masked language modeling, the next step is to load a DistilRoBERTa tokenizer to process the text subfield

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Here is a first preprocessing function to tokenize texts

In [6]:
def preprocess_function(examples):
    return tokenizer(examples['text'])

To apply this preprocessing function over the entire dataset,we use the 🤗 Datasets map method. We can speed up the map function by setting batched=True to process multiple elements of the dataset at once, and increasing the number of processes with num_proc. Remove any columns you don’t need

In [7]:
tokenized_wikitext_ds = wikitext_ds.map(
    preprocess_function,
    batched = True,
    num_proc = 4,
    remove_columns = wikitext_ds['train'].column_names,
)

tokenized_wikitext_ds

Map (num_proc=4):   0%|          | 0/8000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (789 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (843 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (568 > 512). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

This dataset contains the token sequences, but some of these are longer than the maximum input length for the model.

Using  a second preprocessing function to
concatenate all the sequences
split the concatenated sequences into shorter chunks defined by block_size, which should be both shorter than the maximum input length and short enough for your GPU RAM.

In [8]:
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}

    total_length = len(concatenated_examples[list(examples.keys())[0]])

    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size

    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    return result

Applying the group_texts function over the entire dataset

In [9]:
wikitext_lm_dataset = tokenized_wikitext_ds.map(group_texts, batched = True, num_proc = 4)

wikitext_lm_dataset

Map (num_proc=4):   0%|          | 0/8000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4141
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1029
    })
})

Now creating a batch of examples using DataCollatorForLanguageModeling.

each sequence that will be randomly selected for masking. For instance, if mlm_probability is set to 0.15, it means that, on average, 15% of the tokens in each sequence will be replaced with a special [MASK] token.

Use in Masked Language Modeling: MLM is a technique used in training language models like BERT where random tokens in an input sequence are masked and the model is tasked with predicting these masked tokens. The mlm_probability parameter controls how many tokens are masked and thus influences the difficulty and learning dynamics of the MLM task.

Balancing Masking: The choice of mlm_probability is crucial. Too high a value might lead to excessive masking, making the prediction task very difficult and potentially slowing down convergence. Conversely, too low a value might not provide the model with enough masked examples to effectively learn contextual representations.

In [11]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm_probability = 0.15, return_tensors = "tf")

For finetuning a model in TensorFlow, start by setting up an optimizer function, learning rate schedule, and some training hyperparameters

In [12]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

Loading a distilroberta with TFAutoModelForMaskedLM

In [13]:
from transformers import TFAutoModelForMaskedLM

model = TFAutoModelForMaskedLM.from_pretrained("distilroberta-base")

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFRobertaForMaskedLM.

All the weights of TFRobertaForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForMaskedLM for predictions without further training.


Converting  datasets to the tf.data.Dataset format with prepare_tf_dataset()

In [14]:
tf_train_set = model.prepare_tf_dataset(
    wikitext_lm_dataset["train"],
    shuffle = True,
    batch_size = 16,
    collate_fn = data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    wikitext_lm_dataset["test"],
    shuffle = False,
    batch_size = 16,
    collate_fn = data_collator,
)

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Configuring  the model for training with compile. Note that Transformers models all have a default task-relevant loss function, so you don’t need to specify one unless you want to:

In [15]:
import tensorflow as tf

model.compile(optimizer = optimizer)

model.summary()

Model: "tf_roberta_for_masked_lm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLaye  multiple                  81527808  
 r)                                                              
                                                                 
 lm_head (TFRobertaLMHead)   multiple                  39642969  
                                                                 
Total params: 82170201 (313.45 MB)
Trainable params: 82170201 (313.45 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
model.fit(x = tf_train_set, validation_data = tf_test_set, epochs = 3)

Epoch 1/3
258/258 [==============================] - 128s 429ms/step - loss: 2.1543 - val_loss: 1.8564
Epoch 2/3
258/258 [==============================] - 110s 428ms/step - loss: 1.9689 - val_loss: 1.7733
Epoch 3/3
258/258 [==============================] - 109s 423ms/step - loss: 1.8820 - val_loss: 1.7607


Logging into account for pushing trained model to Hub

Like before, we can evaluate our model on the validation set and compute perplexity. The perplexity is much lower than for the CLM objective because for the MLM objective, we only have to make predictions for the masked tokens while having access to the rest of the tokens. It's thus an easier task for the model.

In [17]:
import math

eval_results = model.evaluate(tf_test_set)

print(f"Perplexity: {math.exp(eval_results):.2f}")

65/65 [==============================] - 9s 144ms/step - loss: 1.7519
Perplexity: 5.77


Inferencing our trained model with different instances

In [18]:
#text = "USA plans to <mask> its own space station after 2024"
#text = "i am planning to <mask> this holidays."
#text = "Some examples of wild animals  are <mask> and Tiger."


#text = "Countries are growing <mask> recently"
text= "The economic summit was a meet that took place in <mask> in 2000-2004."

In [20]:
from transformers import pipeline

mask_filler = pipeline("fill-mask", model = model, tokenizer = tokenizer)

mask_filler(text)

[{'score': 0.05746893957257271,
  'token': 5308,
  'token_str': ' Tokyo',
  'sequence': 'The economic summit was a meet that took place in Tokyo in 2000-2004.'},
 {'score': 0.03706979379057884,
  'token': 4612,
  'token_str': ' Barcelona',
  'sequence': 'The economic summit was a meet that took place in Barcelona in 2000-2004.'},
 {'score': 0.03568599745631218,
  'token': 3332,
  'token_str': ' Beijing',
  'sequence': 'The economic summit was a meet that took place in Beijing in 2000-2004.'},
 {'score': 0.03244199976325035,
  'token': 928,
  'token_str': ' London',
  'sequence': 'The economic summit was a meet that took place in London in 2000-2004.'},
 {'score': 0.028203656896948814,
  'token': 6497,
  'token_str': ' Brussels',
  'sequence': 'The economic summit was a meet that took place in Brussels in 2000-2004.'}]